In [1]:
from transformers import pipeline
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sys
sys.path.append("../CODE-Baseline")  
import warnings
warnings.filterwarnings('ignore')


from salary_baseline import extract_salary_with_inference

file_path = '../DATASETS/salary_labelled_development_set.csv'
test_file_path = '../DATASETS/salary_labelled_test_set.csv'
df = pd.read_csv(file_path)

model_name = "deepset/roberta-base-squad2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=-1)

Development dataset:
Precision: 0.7811
Recall: 0.9370
F1 Score: 0.8519
Accuracy: 0.8489

Test dataset:
Precision: 0.7412
Recall: 0.9206
F1 Score: 0.8212
Accuracy: 0.8219


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Device set to use cpu


In [2]:
# html -> context 
def clean_html_tags(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    for tag in soup(["script", "style"]):
        tag.decompose()
    text = soup.get_text(separator=" ", strip=True)
    return text

In [8]:

# 输入问题和上下文
question = "What is the salary?"
context = '''
Protege - Quantity Surveyor (KL) Description Quantity surveyors have under their helm complete management of the cost involved in building and construction projects from the inception of the project until the delivery. They strive for an efficient use of the resources whilst keeping an eye on quality, quality standards, and client's requirements. Contract Basic : 12 months Monthly Salary : RM2000.00 Annual Leave & Medical Leave Provided SOCSO & EIS Contribution *Candidate must be a Bachelor Degree holder* Company PANCARAN DINAR SDN BHD telah ditubuhkan pada tahun 1982. PANCARAN DINAR SDN BHD adalah sebuah Syarikat Bumiputera yang bergiat aktif dalam industry pembinaan di Pantai Timur Semenanjung. Sehingga kini projek-projek yang dikendalikan oleh PANCARAN DINAR SDN BHD merupakan kontrak daripada pihak kerajaan, badan berkanun dan pihak swasta. Jumlah kerja terkumpul yang telah dikendalikan oleh PANCARAN DINAR SDN BHD adalah melebihi RM400 juta merangkumi projek-projek pembinaan bangunan dan infrastruktur. PANCARAN DINAR SDN BHD merupakan sebuah syarikat yang berdaftar dengan Lembaga Pembangunan Industri Pembinaan Malaysia (CIDB) – Kelas G7 dan berdaftar dengan Bahagian Pembangunan Kontraktor Dan Usahawan (BPKU) – Taraf Bumiputera PANCARAN DINAR SDN BHD juga mendapat pengiktirafan ISO 9001:2015 oleh SIRIM QAS International Sdn. Bhd. bagi aktiviti penyediaan perkhidmatan pembinaan untuk bangunan, sivil dan kerja-kerja infrastruktur.

'''
input_text = f"question: {question} context: {context}"

# 生成答案
output = qa_pipeline(input_text)

# 打印生成的答案
print(output)

[{'generated_text': 'question: What is the salary? context: เราร'}]


In [4]:
# salary_question = "what is fixed salary or salary range?"
# pay_freq_question = "Is the salary paid monthly, hourly, yearly, weekly, or daily?"

# def get_salary_using_nFT_Bert(text,nation_code):
#   context=clean_html_tags(text)
#   salary_result = qa_pipeline({"context": context, "question": salary_question})
#   t_result=extract_salary_with_inference(salary_result["answer"],nation_code)
#   # print(t_result)
#   if t_result!='0-0-None-None':
#     pay_freq_result = qa_pipeline({"context": context, "question": pay_freq_question})
#     # print(pay_freq_result["answer"])
    
#   return t_result

    
    
    
# df['predicted_salary'] = df.apply(
#     lambda row: get_salary_using_nFT_Bert(
#         f"{row['job_title']} {row['job_ad_details']}",
#         row['nation_short_desc']
#     ),
#     axis=1
# )
    


In [5]:
# # TP, FP, TN, FN
# TP = np.sum((df['predicted_salary'] == df['y_true']) & (df['y_true'] != "0-0-None-None"))
# FP = np.sum((df['predicted_salary'] != df['y_true']) & (df['predicted_salary'] != "0-0-None-None"))
# FN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] != "0-0-None-None"))
# TN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] == "0-0-None-None"))

# precision = TP / (TP + FP) if (TP + FP) != 0 else 0
# recall = TP / (TP + FN) if (TP + FN) != 0 else 0
# f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
# accuracy = (TP + TN) / (FP + FN + TP + TN)

# # Print prediction vs ground truth
# # print("\n🔍 Prediction vs Ground Truth:\n")
# # for i, row in df.iterrows():
# #     predicted = row['predicted_salary']
# #     expected = row['y_true']
# #     if predicted != expected:
# #         print(f"[{i}] ❌ Predicted: {predicted} | Expected: {expected}")
# #         print(f"{row['job_id']} {row['job_title']} {row['job_ad_details']}")
# #         print()
#     # else:
#     #     print(f"[{i}] ✅ Matched:   {predicted}")

# print("Development dataset:")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")
# print(f"Accuracy: {accuracy:.4f}")
# print()

In [6]:
# # test set
# df = pd.read_csv(test_file_path)
# df['predicted_salary'] = df.apply(
#     lambda row: get_salary_using_nFT_Bert(
#         f"{row['job_title']} {row['job_ad_details']}",
#         row['nation_short_desc']
#     ),
#     axis=1
# )

# TP = np.sum((df['predicted_salary'] == df['y_true']) & (df['y_true'] != "0-0-None-None"))
# FP = np.sum((df['predicted_salary'] != df['y_true']) & (df['predicted_salary'] != "0-0-None-None"))
# FN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] != "0-0-None-None"))
# TN = np.sum((df['predicted_salary'] == "0-0-None-None") & (df['y_true'] == "0-0-None-None"))


# precision = TP / (TP + FP) if (TP + FP) != 0 else 0
# recall = TP / (TP + FN) if (TP + FN) != 0 else 0
# f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
# accuracy = (TP + TN) / (FP + FN + TP + TN)

# print("Test dataset:")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")
# print(f"Accuracy: {accuracy:.4f}")